In [1]:
import pandas as pd
import pickle as pkl
from sklearn.metrics.pairwise import cosine_similarity
import bert
import numpy as np
from bert.extract_features import *

In [2]:
movie_data=pd.read_csv("./movies_metadata.csv")

C:\Users\Dell\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
with open("./emb.pickle","rb") as file:  #loading overview embeddings
    emb=pkl.load(file)

In [4]:
def make_dataset(data):  # extracting data_info
    data['tagline']=data["tagline"].fillna(" ")
    data=data.loc[:,["original_title","genres","overview"]]
    print(data.shape)
    data=data.dropna()
    print(data.shape)
    return data.loc[:,["original_title","genres","overview"]]

data_info=make_dataset(movie_data)

(45466, 3)
(44512, 3)


In [5]:
titles=data_info["original_title"].apply(lambda x:"".join(x.lower().split()))
def get_similar_titles(title):  #return similar titles
    index,j=[],0
    title="".join(title.lower().split())
    for i in titles:
        if title in i:
            index.append(j)
        j+=1
    return data_info.iloc[index]["original_title"],index

In [ ]:
#to extract embeddings from overview

tokenizer = bert.tokenization.FullTokenizer(
  vocab_file="./tmp/wwm_uncased_L-24_H-1024_A-16/vocab.txt", do_lower_case=True)

layer_indexes = [int(x) for x in [-1,-2,-3,-4]]              #last 4 hidden layers

bert_config = bert.modeling.BertConfig.from_json_file("./tmp/wwm_uncased_L-24_H-1024_A-16/bert_config.json")#model configs

tokenizer = bert.tokenization.FullTokenizer(
  vocab_file="./tmp/wwm_uncased_L-24_H-1024_A-16/vocab.txt", do_lower_case=True)

is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2    #
run_config = tf.contrib.tpu.RunConfig(                          #
  master=None,                                                  #    Setting up configurations
  tpu_config=tf.contrib.tpu.TPUConfig(                          #  
      num_shards=None,                                          #
      per_host_input_for_training=is_per_host))                 #   

model_fn = model_fn_builder(                                        #
  bert_config=bert_config,                                          #
  init_checkpoint="./tmp/wwm_uncased_L-24_H-1024_A-16/bert_model.ckpt",  #  funtion to build pretrained model
  layer_indexes=layer_indexes,                                      #
  use_tpu=False,                                                    # 
  use_one_hot_embeddings=False)                                     #

estimator = tf.contrib.tpu.TPUEstimator(                            #
  use_tpu=False,                                                    #
  model_fn=model_fn,                                                # predictor to extract features 
  config=run_config,                                                #
  predict_batch_size=12)

In [6]:
def extract_feat(text): #extract embeddings
    
    text_=[InputExample(unique_id=0, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = text, 
                                                                   text_b = None)]
    train_data=convert_examples_to_features(text_
                                        ,256,tokenizer,) #converting to features expected by the model
    input_fn = input_fn_builder(features=train_data, seq_length=256)   #prepares input
    
    for result in estimator.predict(input_fn, yield_single_examples=True):
        l1=np.mean(axis=0,a=result['layer_output_0'])     #
        l2=np.mean(axis=0,a=result['layer_output_1'])     # Last 4 hidden layers features
        l3=np.mean(axis=0,a=result['layer_output_2'])     # 
        l4=np.mean(axis=0,a=result['layer_output_3'])     #

        emb=np.mean(axis=0,a=[l1,l2,l3,l4])               # taking mean to get sentence embeddings
        return emb                                                  # saving with movie titles
    return -1      

W0903 23:32:44.480884 14688 deprecation_wrapper.py:119] From C:\Users\Dell\Anaconda3\lib\site-packages\bert\tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0903 23:32:46.163624 14688 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0903 23:32:46.165508 14688 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x000001F2D62B5488>) includes params argument, but params are not passed to Estimator.
W0903 23:32:46.169753 14688 estimator.py:1811] Using temporary folder as model directory: C:\Users\Dell\AppData\Local\Temp\tmp0t20uylw
W0903 23:32:46.172525 14688 tpu_context.py

In [7]:
def get_nearest_movie(title,n=10):     #return 10 similar movies based on cosine similarity
    rec,rec_movies=[],[]
    rec_movies.append((title,data_info[data_info["original_title"]==title]["overview"]))
    vec=emb[title]
    for i in emb.keys():
        if i == title:
            continue
        x=cosine_similarity(vec.reshape(1,-1),emb[i].reshape(1,-1))
        rec.append((x,i))
    for j,i in sorted(rec,key=lambda x : x[0],reverse=True)[:n]:
        rec_movies.append((i,data_info[data_info["original_title"]==i]["overview"]))
    return rec_movies 

In [8]:
def get_nearest_overview(text,n=10):  #return 10 similar movies based on given overview
    vec=extract_feat(text)
    rec_movies,rec=[],[]
    for i in emb.keys():
        x=cosine_similarity(vec.reshape(1,-1),emb[i].reshape(1,-1))
        rec.append((x,i))
    for j,i in sorted(rec,key=lambda x : x[0],reverse=True)[:n]:
        rec_movies.append((i,data_info[data_info["original_title"]==i]["overview"],j))
    return rec_movies 

In [14]:
ans=get_nearest_overview("Terrorist organization attack the world") #get similar movies based on the given overview
for i in ans:
    print(i[0],":",i[1].iloc[0])
    print()

The Batman Superman Movie: World's Finest : Joker goes to Metropolis with an offer and plan to kill Superman for Lex Luthor while Batman pursues the clown to Superman's turf

The Mad Scientist : The Man of Steel fights a mad scientist who is destroying Metropolis with an energy cannon.

DC Showcase: Catwoman : Catwoman attempts to track down a mysterious cargo shipment that is linked to a Gotham City crime boss called Rough Cut.

Short Circuit 2 : Robot Johnny Five comes to the city and gets manipulated by criminals who want him for their own purposes.

Coogan's Bluff : Coogan (Clint Eastwood) an Arizona deputy sheriff goes to New York to pick up a prisoner. While escorting the prisoner to the airport he escapes and Coogan heads into the City to recapture him.

Squadra antimafia : Undercover cop Nico Giraldi travels to New York and Las Vegas to find a crooked cop who gave his squadron back in Italy a bad name.

Company Business : An aging agent is called back by "the Company" to run a 

In [10]:
names,index=get_similar_titles("Batman  Superman")  #get names of the movies containing the given title
names

9169    The Batman Superman Movie: World's Finest
Name: original_title, dtype: object

In [15]:
ans=get_nearest_movie("The Batman Superman Movie: World's Finest") #get similar movies to the given one
for i in ans:
    print(i[0],":",i[1].iloc[0])
    print()

The Batman Superman Movie: World's Finest : Joker goes to Metropolis with an offer and plan to kill Superman for Lex Luthor while Batman pursues the clown to Superman's turf

The Mad Scientist : The Man of Steel fights a mad scientist who is destroying Metropolis with an energy cannon.

DC Showcase: Catwoman : Catwoman attempts to track down a mysterious cargo shipment that is linked to a Gotham City crime boss called Rough Cut.

Short Circuit 2 : Robot Johnny Five comes to the city and gets manipulated by criminals who want him for their own purposes.

Coogan's Bluff : Coogan (Clint Eastwood) an Arizona deputy sheriff goes to New York to pick up a prisoner. While escorting the prisoner to the airport he escapes and Coogan heads into the City to recapture him.

Squadra antimafia : Undercover cop Nico Giraldi travels to New York and Las Vegas to find a crooked cop who gave his squadron back in Italy a bad name.

Company Business : An aging agent is called back by "the Company" to run a 